In [2]:
pip install streamlit pandas plotly openpyxl

  Using cached streamlit-1.45.1-py3-none-any.whl (9.9 MB)
     |████████████████████████████████| 16.3 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 250 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 17.6 MB/s eta 0:00:01
     |████████████████████████████████| 425 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 207 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 731 kB 14.3 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 18.1 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 7.9 MB/s  eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to res

In [3]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

In [4]:
# 1. 数据加载
@st.cache_data
def load_data():
    obs = pd.read_excel("/Users/lvlei/PycharmProjects/pythonProject3/Data/New_Area_Working_Data.xlsx")
    params = pd.read_excel("/Users/lvlei/PycharmProjects/pythonProject3/Data/Physiological parameters and algorithm table.xlsx")
    return obs, params

obs, params = load_data()

# 2. Care Home选择
care_homes = obs['CareHome'].unique()
carehome_choice = st.selectbox("Choose Care Home", care_homes)

# 筛选数据
obs_ch = obs[obs['CareHome'] == carehome_choice].copy()

# 3. 时间区间、粒度选择
time_gran = st.selectbox("Time Interval", ['Daily', 'Weekly', 'Monthly', 'Yearly'])
obs_ch['Date'] = pd.to_datetime(obs_ch['Date'])
obs_ch = obs_ch.sort_values('Date')

# 4. Usage Analysis
st.header("Usage Analysis")

# (1) usage count 按粒度重采样
if time_gran == 'Daily':
    usage = obs_ch.groupby(obs_ch['Date'].dt.date).size()
elif time_gran == 'Weekly':
    usage = obs_ch.resample('W-MON', on='Date').size()
elif time_gran == 'Monthly':
    usage = obs_ch.resample('M', on='Date').size()
else:
    usage = obs_ch.resample('Y', on='Date').size()
usage = usage.reset_index()
usage.columns = ['Date', 'Count']

fig = px.line(usage, x='Date', y='Count', markers=True, title=f"{time_gran} Usage Count")
st.plotly_chart(fig, use_container_width=True)

# (2) usage per bed
beds = obs_ch['Beds'].iloc[0] if 'Beds' in obs_ch.columns else st.number_input("Input Number of Beds")
usage['Usage_per_bed'] = usage['Count'] / beds
fig2 = px.line(usage, x='Date', y='Usage_per_bed', markers=True, title=f"{time_gran} Usage per Bed")
st.plotly_chart(fig2, use_container_width=True)

# (3) Monthly Observation coverage percentage
if time_gran == 'Monthly':
    obs_ch['Month'] = obs_ch['Date'].dt.to_period('M')
    cov = obs_ch.groupby('Month')['Date'].nunique().reset_index()
    cov['Days_in_Month'] = cov['Month'].dt.days_in_month
    cov['Coverage%'] = cov['Date'] / cov['Days_in_Month'] * 100
    fig3 = px.line(cov, x='Month', y='Coverage%', markers=True, title="Monthly Coverage Percentage")
    st.plotly_chart(fig3, use_container_width=True)

# ======================
# 下面是 Clinical Insight Value
st.header("Clinical Insight Value")

# (1) Monthly counts for each NEWS2
obs_ch['Month'] = obs_ch['Date'].dt.to_period('M')
news_counts = obs_ch.groupby(['Month', 'NEWS2_Score']).size().reset_index(name='Counts')
fig4 = px.line(news_counts, x='Month', y='Counts', color='NEWS2_Score', markers=True, title="Monthly NEWS2 Counts")
st.plotly_chart(fig4, use_container_width=True)

# (2) Monthly proportion of high-risk scores (NEWS2>=6)
obs_ch['High_Risk'] = obs_ch['NEWS2_Score'] >= 6
high_risk = obs_ch.groupby('Month')['High_Risk'].mean().reset_index()
high_risk['High_Risk%'] = high_risk['High_Risk'] * 100
fig5 = px.line(high_risk, x='Month', y='High_Risk%', markers=True, title="Monthly High-Risk (NEWS2≥6) %")
st.plotly_chart(fig5, use_container_width=True)

# (3) concern-triggered比例
if 'Concern' in obs_ch.columns:
    obs_ch['Concern_Y'] = obs_ch['Concern'] == "Yes"
    concern_rate = obs_ch.groupby('Month')['Concern_Y'].mean().reset_index()
    concern_rate['Concern%'] = concern_rate['Concern_Y'] * 100
    fig6 = px.line(concern_rate, x='Month', y='Concern%', markers=True, title="Monthly Concern-triggered %")
    st.plotly_chart(fig6, use_container_width=True)

# (4) clinical judgement准确率
if 'Concern_Y' in obs_ch.columns:
    both = obs_ch[(obs_ch['Concern_Y']) & (obs_ch['High_Risk'])]
    accuracy = both.groupby('Month').size() / obs_ch[obs_ch['Concern_Y']].groupby('Month').size()
    acc_df = accuracy.reset_index(name='Accuracy')
    acc_df['Accuracy%'] = acc_df['Accuracy'] * 100
    fig7 = px.line(acc_df, x='Month', y='Accuracy%', markers=True, title="Monthly Staff Clinical Judgement Accuracy")
    st.plotly_chart(fig7, use_container_width=True)

# (5) Parameters trigger high score
param_cols = [c for c in obs_ch.columns if '_New' in c and c != 'NEWS2_Score']
high_score = obs_ch[obs_ch['NEWS2_Score'] >= 6]
param_trig = high_score[param_cols].apply(lambda x: (x>=1).sum(), axis=0)  # 自定义逻辑可微调
param_trig = param_trig.reset_index()
param_trig.columns = ['Parameter', 'Count']
fig8 = px.bar(param_trig, x='Parameter', y='Count', title="Which Physiological Parameters Trigger High Scores")
st.plotly_chart(fig8, use_container_width=True)

2025-06-06 06:30:58.935 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-06 06:30:58.937 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-06-06 06:30:58.939 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 06:30:59.090 
  command:

    streamlit run /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-06-06 06:30:59.090 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 06:30:59.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 06:30:59.598 Thread 'Thread-5': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-06 06:30:5

KeyError: 'CareHome'